In [15]:
#read the training file
train_corpus = []
with open('train.txt',encoding="utf8") as readFile:
    for text in readFile:
        train_corpus.append(text.strip())

In [16]:
#read the testing file
test_corpus = []
with open('test.txt',encoding="utf8") as readFile:
    for text in readFile:
        test_corpus.append(text.strip())

In [17]:
#Unigram Model
unigram_dist = {}
for text in train_corpus:
    text = text.lower() + ' </s>'
    for word in text.split():
        if word in unigram_dist:
            unigram_dist[word] += 1
        else:
            unigram_dist[word] = 1

In [19]:
# 1.3  Q 1:
count = 0
unk_flag = False
for word in unigram_dist:
    if unigram_dist[word] == 1:
        unk_flag = True
    else:
        count += 1
if unk_flag:
    count += 1
print('Word Types(Unique words) in training Corpus: {}'.format(count))

Word Types(Unique words) in training Corpus: 41738


In [20]:
# 1.3  Q 2:
count = 0
for word in unigram_dist:
    count += unigram_dist[word]
print('Word Tokens in training Corpus: {}'.format(count))

Word Tokens in training Corpus: 2468210


In [21]:
# 1.3 Q 3
words_in_training = []
words_not_in_training = []
for text in test_corpus:
    text = text.lower() + ' </s>'
    for word in text.split():
        if word in unigram_dist:
            words_in_training.append(word)
        else:
            words_not_in_training.append(word)
print('No I did not mapped the unknown words to <unk> in training and testing data')
print('Word types percentage did not occur in training: {} %'.format((1-(len(set(words_in_training))/(len(set(words_in_training))+len(set(words_not_in_training)))))*100))
print('Word tokens percentage did not occur in training: {} %'.format((1-(len(words_in_training)/(len(words_in_training)+len(words_not_in_training))))*100))

No I did not mapped the unknown words to <unk> in training and testing data
Word types percentage did not occur in training: 3.605769230769229 %
Word tokens percentage did not occur in training: 1.6612495485734957 %


In [22]:
#replace the singletons into <unk>
update_unigram_dist = {}
for word in unigram_dist:
    if unigram_dist[word] == 1:
        if '<unk>' not in update_unigram_dist:
            update_unigram_dist['<unk>'] = 0
        else:
            update_unigram_dist['<unk>'] += 1
    else:
        update_unigram_dist[word] = unigram_dist[word]

In [23]:

bigram_dist = {}
for text in train_corpus:
    text = text.lower() + ' </s>'
    tokens = text.split()
    for i in range(1,len(tokens)):
        w0 = tokens[i-1]
        w1 = tokens[i]
        if w0 not in update_unigram_dist:
            w0 = '<unk>'
        if w1 not in update_unigram_dist:
            w1 = '<unk>'
        if (w0,w1) in bigram_dist:
            bigram_dist[(w0,w1)] += 1
        else:
            bigram_dist[(w0,w1)] = 1

In [24]:
# 1.3 Q 4
bigrams_in_training = []
bigrams_not_in_training = []
for text in test_corpus:
    text = text.lower() + ' </s>'
    tokens = text.split()
    for i in range(1,len(tokens)):
        w0 = tokens[i-1]
        w1 = tokens[i]
        if w0 not in update_unigram_dist:
            w0 = '<unk>'
        if w1 not in update_unigram_dist:
            w1 = '<unk>'
        if (w0,w1) in bigram_dist:
            bigrams_in_training.append((w0,w1))
        else:
            bigrams_not_in_training.append((w0,w1))
print('Bigrams types percentage did not occur in training: {} %'.format((1-(len(set(bigrams_in_training))/(len(set(bigrams_in_training))+len(set(bigrams_not_in_training)))))*100))
print('Bigrams tokens percentage did not occur in training: {} %'.format((1-(len(bigrams_in_training)/(len(bigrams_in_training)+len(bigrams_not_in_training))))*100))

Bigrams types percentage did not occur in training: 25.869565217391298 %
Bigrams tokens percentage did not occur in training: 22.36792806294492 %


In [25]:
unigramN = 0
for word in update_unigram_dist:
    unigramN += update_unigram_dist[word]

In [26]:
# 1.3  Q 5:
import numpy as np
def unigramLogProb(text):
    tokens = (text.lower()+' </s>').split()
    log_value = 0
    token_list = []
    for token in tokens:
        if token not in update_unigram_dist:
            token = '<unk>'
        prob = update_unigram_dist[token]/unigramN
        value = np.log2(prob)
        token_list.append((token,prob))
        log_value += value
    return log_value, token_list
text = "I look forward to hearing your reply"
log_prob, token_list = unigramLogProb(text)
for token in token_list:
    print(token)
print('\nLog probability: {}'.format(log_prob))

('i', 0.0029734110847177042)
('look', 0.0002483582225006067)
('forward', 0.00019204208395642345)
('to', 0.02149250732008513)
('hearing', 8.467678385420359e-05)
('your', 0.0004930700763184965)
('reply', 5.266976986146635e-06)
('</s>', 0.04051520758574335)

Log probability: -84.9288472425838


In [27]:
def bigramLogProb(text):
    tokens = (text.lower()+' </s>').split()
    log_value = 0
    token_list = []
    for i in range(1,len(tokens)):
        w0 = tokens[i-1]
        w1 = tokens[i]
        if w0 not in update_unigram_dist:
            w0 = '<unk>'
        if w1 not in update_unigram_dist:
            w1 = '<unk>'
        if (w0,w1) not in bigram_dist:
            bigram_occur = 0
        else:
            bigram_occur = bigram_dist[(w0,w1)]
        prob = bigram_occur/update_unigram_dist[w0]
        value = np.log2(prob)
        token_list.append(((tokens[i-1],tokens[i]),prob))
        log_value += value
    return log_value, token_list
text = "I look forward to hearing your reply"
log_prob, token_list = bigramLogProb(text)
for token in token_list:
    print(token)
print('\nLog probability: {}'.format(log_prob))

(('i', 'look'), 0.0020438751873552256)
(('look', 'forward'), 0.05546492659053834)
(('forward', 'to'), 0.2109704641350211)
(('to', 'hearing'), 0.00011310511235107827)
(('hearing', 'your'), 0.0)
(('your', 'reply'), 0.0)
(('reply', '</s>'), 0.0)

Log probability: -inf


/home/raza/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:17: RuntimeWarning: divide by zero encountered in log2


In [28]:
def bigramLogProbAddSmooth(text):
    tokens = (text.lower()+' </s>').split()
    log_value = 0
    token_list = []
    for i in range(1,len(tokens)):
        w0 = tokens[i-1]
        w1 = tokens[i]
        if w0 not in update_unigram_dist:
            w0 = '<unk>'
        if w1 not in update_unigram_dist:
            w1 = '<unk>'
        if (w0,w1) not in bigram_dist:
            bigram_occur = 0
        else:
            bigram_occur = bigram_dist[(w0,w1)]
        prob = (bigram_occur+1)/(update_unigram_dist[w0]+len(update_unigram_dist))
        value = np.log2(prob)
        token_list.append(((tokens[i-1],tokens[i]),prob))
        log_value += value
    return log_value, token_list
text = "I look forward to hearing your reply"
log_prob, token_list = bigramLogProbAddSmooth(text)
for token in token_list:
    print(token)
print('\nLog probability: {}'.format(log_prob))

(('i', 'look'), 0.0003260182977769627)
(('look', 'forward'), 0.0008264267667823664)
(('forward', 'to'), 0.0023926845446792383)
(('to', 'hearing'), 7.38505686493786e-05)
(('hearing', 'your'), 2.3839607123274607e-05)
(('your', 'reply'), 2.3280176929344664e-05)
(('reply', '</s>'), 2.3951522119230678e-05)

Log probability: -90.35211196443383


In [29]:
def unigramPerplexity(corpus):
    sum = 0
    N = 0
    for text in corpus:
        words = text.split()
        N = N + len(words)+1 # +1 </s> tags
        sum = sum + unigramLogProb(text)[0]
    pp = np.power(2, -(sum/N))
    return pp

In [30]:
def bigramPerplexity(corpus):
    sum = 0
    N = 0
    for text in corpus:
        words = text.split()
        N = N + len(words)+1 # +1 </s> tags
        sum = sum + bigramLogProb(text)[0]
    pp = np.power(2, -(sum/N))
    return pp

In [31]:
def bigramPerplexityAddSmooth(corpus):
    sum = 0
    N = 0
    for text in corpus:
        words = text.split()
        N = N + len(words)+1 # +1 </s> tags
        sum = sum + bigramLogProbAddSmooth(text)[0]
    pp = np.power(2, -(sum/N))
    return pp

In [33]:
# 1.3  Q 6:
text = "I look forward to hearing your reply"
print(unigramPerplexity([text]))
print(bigramPerplexity([text]))
print(bigramPerplexityAddSmooth([text]))

1569.5180501674179
inf
2510.9437518832056


/home/raza/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:17: RuntimeWarning: divide by zero encountered in log2


In [35]:
# 1.3  Q 7:
print(unigramPerplexity(test_corpus))
print(bigramPerplexity(test_corpus))
print(bigramPerplexityAddSmooth(test_corpus))

1097.1906220380547
inf
1905.9287857229451


/home/raza/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:17: RuntimeWarning: divide by zero encountered in log2
